In [1]:
# Dependencies
from bs4 import BeautifulSoup
import pandas as pd
import requests
import warnings
from splinter import Browser
import time
import os
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/kscomputer/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


NASA Mars News

In [3]:
url1 = 'https://mars.nasa.gov/news/'
browser.visit(url1)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
title = soup.find('div',class_ = 'content_title').text
para = soup.find('div',class_ = 'article_teaser_body').text

print(title)
print(para)

Mars Now
The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative.


JPL Mars Space Images - Featured Image

In [5]:
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
image_url = soup.find("a",class_="button fancybox").get("data-fancybox-href")

base_url = 'https://www.jpl.nasa.gov'

featured_url = base_url + image_url

featured_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17563_ip.jpg'

Mars Facts

In [7]:
url3 = 'https://space-facts.com/mars/'

In [8]:
tables = pd.read_html(url3)

In [9]:
df = tables[0]
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [10]:
table = df.to_html(classes = 'table')
print(table)

<table border="1" class="dataframe table">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Re

Mars Hemispheres

In [11]:
url4 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url4)

In [12]:
hemispheres = []

for i in range (4):
    
    browser.find_by_tag('h3')[i].click()
    
    html = browser.html
    
    soup = BeautifulSoup(html, 'html.parser')
    
    image = soup.find('img', class_='wide-image')['src']
    
    image_url = 'https://astrogeology.usgs.gov' + image
    
    title = soup.find('h2', class_= 'title').text 
    
    hemispheres.append({'Title': title, 'Image URL':image_url})

In [13]:
for i in (hemispheres):
    print(i)

{'Title': 'Cerberus Hemisphere Enhanced', 'Image URL': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}
{'Title': 'Cerberus Hemisphere Enhanced', 'Image URL': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}
{'Title': 'Cerberus Hemisphere Enhanced', 'Image URL': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}
{'Title': 'Valles Marineris Hemisphere Unenhanced', 'Image URL': 'https://astrogeology.usgs.gov/cache/images/5277a23c6d8a3a325985965390cb77a2_valles_marineris_unenhanced.tif_full.jpg'}


In [14]:
browser.quit()